# AMI Corpus - Démonstration SDM et Segmentation des Locuteurs

Ce notebook démontre l'utilisation du corpus AMI avec :
- **Configuration SDM** (Single Distant Microphone - Microphone distant unique)
- **Segmentation des locuteurs** automatique
- **Visualisation interactive** avec lecture audio
- **Exemple multi-locuteurs** complet

## À propos de la configuration SDM
- **SDM** = Single Distant Microphone
- Utilise un **seul microphone distant** pour l'enregistrement
- Configuration plus simple que MDM (Multiple Distant Microphone)
- Idéale pour tester les algorithmes de diarisation sur signal mono

In [ ]:
# Installation des dépendances avec gestion de compatibilité NumPy
!pip install "numpy<2" --force-reinstall
!pip install "datasets<4.0.0"
!pip install  librosa soundfile matplotlib ipython scikit-learn
print("⚠️  Si des erreurs de compatibilité persistent, redémarrez le kernel après l'installation")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from datasets import load_dataset
from IPython.display import Audio, display, HTML
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

print("✅ Toutes les librairies importées avec succès")

## 1. Chargement du Corpus AMI - Configuration SDM

In [ ]:
def load_ami_sdm(split: str = "train", max_samples: int = 1) -> Dict:
    """
    Charge le corpus AMI avec configuration SDM (Single Distant Microphone)
    """
    print("🎯 Chargement du corpus AMI - Configuration SDM (Single Distant Microphone)...")
    
    try:
        # Charger avec la configuration SDM spécifiquement
        print("🔄 Tentative avec configuration 'sdm' (Single Distant Microphone)...")
        dataset = load_dataset("edinburghcstr/ami", "sdm", split=f"{split}[:{max_samples}]")
        print("✅ Configuration 'sdm' chargée avec succès")
        
    except Exception as e1:
        print(f"⚠️  Configuration 'sdm' échouée: {str(e1)[:100]}...")
        
        try:
            # Essayer avec streaming=True pour contourner certains problèmes
            print("🔄 Tentative SDM avec streaming=True...")
            dataset = load_dataset("edinburghcstr/ami", "sdm", split=f"{split}[:{max_samples}]", streaming=True)
            # Convertir en liste pour l'usage normal
            dataset = list(dataset)
            
            # Créer un objet compatible
            class StreamDataset:
                def __init__(self, data_list):
                    self.data = data_list
                    if data_list:
                        self.column_names = list(data_list[0].keys())
                    else:
                        self.column_names = []
                
                def __len__(self):
                    return len(self.data)
                
                def __getitem__(self, idx):
                    return self.data[idx]
            
            dataset = StreamDataset(dataset)
            print("✅ Dataset SDM en streaming chargé avec succès")
            
        except Exception as e2:
            print(f"⚠️  Streaming SDM échoué: {str(e2)[:100]}...")
            
            try:
                # Essayer de charger depuis le cache local s'il existe
                print("🔄 Tentative SDM avec cache local...")
                import os
                cache_dir = os.path.expanduser("~/.cache/huggingface/datasets/edinburghcstr___ami")
                
                if os.path.exists(cache_dir):
                    print(f"📁 Cache trouvé dans {cache_dir}")
                    dataset = load_dataset("edinburghcstr/ami", "sdm", split=f"{split}[:{max_samples}]", cache_dir=cache_dir)
                else:
                    raise Exception("Cache non trouvé")
                
            except Exception as e3:
                print(f"❌ Toutes les tentatives SDM ont échoué")
                print(f"Dernière erreur: {str(e3)[:200]}...")
                print("\n💡 SOLUTIONS POSSIBLES:")
                print("1. Vérifiez votre connexion internet")
                print("2. Installez huggingface-cli: !pip install huggingface_hub")
                print("3. Téléchargez manuellement: !huggingface-cli download edinburghcstr/ami --repo-type dataset")
                print("4. Le dataset SDM pourrait être temporairement indisponible")
                
                raise Exception("Impossible de charger le corpus AMI avec configuration SDM. Vérifiez les solutions ci-dessus.")
    
    print(f"✅ Dataset SDM chargé: {len(dataset)} échantillons")
    print(f"📊 Colonnes disponibles: {dataset.column_names}")
    
    return dataset

# Charger un échantillon pour tester
dataset = load_ami_sdm(split="train", max_samples=1)
sample = dataset[0]

print("\n🔍 Structure de l'échantillon SDM:")
for key, value in sample.items():
    if key == 'audio':
        print(f"  {key}: array shape = {np.array(value['array']).shape}, sr = {value['sampling_rate']}")
    elif isinstance(value, list):
        print(f"  {key}: liste de {len(value)} éléments")
    else:
        print(f"  {key}: {type(value).__name__}")

## 2. Vérification de l'Audio SDM (Single Distant Microphone)

In [ ]:
def analyze_sdm_audio(sample: Dict) -> Dict:
    """
    Analyse la configuration SDM (Single Distant Microphone)
    """
    audio_data = np.array(sample['audio']['array'])
    sr = sample['audio']['sampling_rate']
    
    analysis = {
        'shape': audio_data.shape,
        'num_channels': 1,  # SDM est toujours mono
        'num_samples': len(audio_data) if len(audio_data.shape) == 1 else audio_data.shape[-1],
        'duration_seconds': (len(audio_data) if len(audio_data.shape) == 1 else audio_data.shape[-1]) / sr,
        'sampling_rate': sr,
        'is_mono': len(audio_data.shape) == 1,
        'configuration': 'SDM'
    }
    
    return analysis, audio_data

# Analyser l'audio SDM
audio_analysis, audio_array = analyze_sdm_audio(sample)

print("🎙️ ANALYSE AUDIO SDM (Single Distant Microphone):")
print(f"   📐 Forme des données: {audio_analysis['shape']}")
print(f"   🎚️ Configuration: {audio_analysis['configuration']}")
print(f"   🔊 Canaux: {audio_analysis['num_channels']} (mono)")
print(f"   ⏱️ Durée: {audio_analysis['duration_seconds']:.1f} secondes")
print(f"   📊 Fréquence d'échantillonnage: {audio_analysis['sampling_rate']} Hz")
print(f"   ✅ Mono: {'✅' if audio_analysis['is_mono'] else '❌'}")

# Calcul des statistiques audio
audio_rms = np.sqrt(np.mean(audio_array**2))
audio_max = np.max(np.abs(audio_array))
audio_energy = np.sum(audio_array**2)

print(f"\n📈 Statistiques du signal SDM:")
print(f"   RMS: {audio_rms:.4f}")
print(f"   Amplitude max: {audio_max:.4f}")
print(f"   Énergie totale: {audio_energy:.2e}")

if not audio_analysis['is_mono']:
    print("\n⚠️  Attention: Le signal n'est pas mono comme attendu pour SDM")

## 3. Segmentation des Locuteurs

In [ ]:
def analyze_speaker_segmentation(sample: Dict) -> Dict:
    """
    Analyse la segmentation des locuteurs (utilise les annotations si disponibles)
    """
    # Vérifier si les segments sont disponibles
    has_segments = all(key in sample for key in ['segment_start_times', 'segment_end_times', 'segment_speakers'])
    
    if has_segments:
        print("✅ Segmentation disponible dans les annotations")
        
        segmentation_info = {
            'source': 'annotations',
            'total_segments': len(sample['segment_speakers']),
            'unique_speakers': list(set(sample['segment_speakers'])),
            'num_speakers': len(set(sample['segment_speakers'])),
            'start_times': sample['segment_start_times'],
            'end_times': sample['segment_end_times'],
            'speakers': sample['segment_speakers'],
            'total_duration': sample['segment_end_times'][-1] - sample['segment_start_times'][0],
            'segments_per_speaker': {}
        }
        
        # Compter segments par locuteur
        for speaker in segmentation_info['unique_speakers']:
            count = sample['segment_speakers'].count(speaker)
            segmentation_info['segments_per_speaker'][speaker] = count
            
    else:
        print("⚠️  Pas de segmentation dans les annotations - Création de segments factices pour la démo")
        
        # Créer une segmentation factice pour la démonstration
        duration = audio_analysis['duration_seconds']
        num_segments = min(8, int(duration / 5))  # Un segment toutes les 5 secondes max
        
        speakers = [f'Speaker_{i%3 + 1}' for i in range(num_segments)]  # 3 locuteurs fictifs
        segment_duration = duration / num_segments
        
        start_times = [i * segment_duration for i in range(num_segments)]
        end_times = [(i + 1) * segment_duration for i in range(num_segments)]
        
        segmentation_info = {
            'source': 'synthetic',
            'total_segments': num_segments,
            'unique_speakers': list(set(speakers)),
            'num_speakers': len(set(speakers)),
            'start_times': start_times,
            'end_times': end_times,
            'speakers': speakers,
            'total_duration': duration,
            'segments_per_speaker': {speaker: speakers.count(speaker) for speaker in set(speakers)}
        }
    
    return segmentation_info

# Analyser la segmentation
seg_info = analyze_speaker_segmentation(sample)

print(f"\n🎯 SEGMENTATION DES LOCUTEURS ({seg_info['source']}):")
print(f"   📊 Nombre de segments: {seg_info['total_segments']}")
print(f"   👥 Nombre de locuteurs: {seg_info['num_speakers']}")
print(f"   🏷️ Locuteurs: {seg_info['unique_speakers']}")
print(f"   ⏱️ Durée totale: {seg_info['total_duration']:.1f}s")
print(f"   📈 Segments par locuteur: {seg_info['segments_per_speaker']}")

## 4. Visualisation Interactive avec Lecture Audio

In [ ]:
def create_sdm_interactive_demo(audio_array: np.ndarray, seg_info: Dict, sr: int, max_duration: float = 60.0):
    """
    Crée une démonstration interactive pour SDM avec visualisation et lecture audio
    """
    # Assurer que nous avons un signal mono pour SDM
    if len(audio_array.shape) > 1:
        audio_array = audio_array.flatten()
    
    # Limiter à la durée maximale
    max_samples = int(max_duration * sr)
    if len(audio_array) > max_samples:
        audio_array = audio_array[:max_samples]
        actual_duration = max_duration
    else:
        actual_duration = len(audio_array) / sr
    
    # Créer la visualisation SDM
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # 1. Timeline des locuteurs
    ax1 = axes[0]
    speakers = seg_info['speakers']
    start_times = seg_info['start_times']
    end_times = seg_info['end_times']
    
    # Couleurs pour chaque locuteur
    unique_speakers = seg_info['unique_speakers']
    colors = plt.cm.Set3(np.linspace(0, 1, len(unique_speakers)))
    speaker_colors = {speaker: colors[i] for i, speaker in enumerate(unique_speakers)}
    
    # Dessiner les segments
    for i, (speaker, start, end) in enumerate(zip(speakers, start_times, end_times)):
        if start > actual_duration:
            break
        end = min(end, actual_duration)
        ax1.barh(speaker, end - start, left=start, 
                color=speaker_colors[speaker], alpha=0.7, 
                edgecolor='black', linewidth=0.5)
        
        # Ajouter le numéro du segment
        ax1.text(start + (end-start)/2, speaker, f'{i+1}', 
                ha='center', va='center', fontweight='bold')
    
    ax1.set_xlabel('Temps (secondes)')
    ax1.set_ylabel('Locuteurs')
    ax1.set_title('Timeline de la Segmentation des Locuteurs - Configuration SDM')
    ax1.set_xlim(0, actual_duration)
    ax1.grid(True, alpha=0.3)
    
    # 2. Forme d'onde mono SDM
    ax2 = axes[1]
    time_axis = np.linspace(0, actual_duration, len(audio_array))
    
    ax2.plot(time_axis, audio_array, label='Signal SDM (mono)', alpha=0.8, color='blue')
    ax2.set_xlabel('Temps (secondes)')
    ax2.set_ylabel('Amplitude')
    ax2.set_title('Forme d\'onde SDM (Single Distant Microphone)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Ajouter l'enveloppe du signal
    window_size = max(1, len(audio_array) // 1000)
    envelope = np.abs(audio_array)
    if len(envelope) > window_size:
        envelope = np.convolve(envelope, np.ones(window_size)/window_size, mode='same')
    ax2.plot(time_axis, envelope, label='Enveloppe', alpha=0.6, color='red', linewidth=2)
    ax2.plot(time_axis, -envelope, alpha=0.6, color='red', linewidth=2)
    ax2.legend()
    
    # 3. Spectrogramme SDM
    ax3 = axes[2]
    
    # Calculer le spectrogramme
    D = librosa.stft(audio_array, hop_length=512)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    
    img = librosa.display.specshow(S_db, sr=sr, hop_length=512, 
                                  x_axis='time', y_axis='hz', ax=ax3)
    ax3.set_title('Spectrogramme SDM (Single Distant Microphone)')
    fig.colorbar(img, ax=ax3, format='%+2.0f dB')
    
    plt.tight_layout()
    plt.show()
    
    return actual_duration

# Créer la visualisation SDM
actual_duration = create_sdm_interactive_demo(audio_array, seg_info, audio_analysis['sampling_rate'])

## 5. Lecture Audio par Canal et par Locuteur

In [ ]:
def create_sdm_audio_players(audio_array: np.ndarray, seg_info: Dict, sr: int, max_duration: float = 60.0):
    """
    Crée des lecteurs audio pour SDM et extrait des locuteurs
    """
    # Assurer que nous avons un signal mono pour SDM
    if len(audio_array.shape) > 1:
        audio_array = audio_array.flatten()
    
    # Limiter la durée
    max_samples = int(max_duration * sr)
    if len(audio_array) > max_samples:
        audio_array = audio_array[:max_samples]
    
    print("? LECTEURS AUDIO SDM:")
    
    # 1. Lecture du signal SDM complet
    print("\n?️ Signal SDM complet (Single Distant Microphone):")
    display(Audio(audio_array, rate=sr))
    
    # 2. Extraits par locuteur
    print("\n👥 EXTRAITS PAR LOCUTEUR (depuis signal SDM):")
    
    for speaker in seg_info['unique_speakers']:
        print(f"\n🎤 {speaker}:")
        
        # Extraire tous les segments de ce locuteur
        speaker_audio = []
        segment_count = 0
        total_speaker_duration = 0
        
        for i, (seg_speaker, start_time, end_time) in enumerate(
            zip(seg_info['speakers'], seg_info['start_times'], seg_info['end_times'])):
            
            if seg_speaker == speaker and start_time < max_duration:
                start_sample = int(start_time * sr)
                end_sample = int(min(end_time, max_duration) * sr)
                
                # S'assurer que les indices sont valides
                start_sample = max(0, min(start_sample, len(audio_array)-1))
                end_sample = max(start_sample+1, min(end_sample, len(audio_array)))
                
                segment_audio = audio_array[start_sample:end_sample]
                speaker_audio.extend(segment_audio)
                segment_count += 1
                total_speaker_duration += (end_sample - start_sample) / sr
        
        if speaker_audio and len(speaker_audio) > 0:
            speaker_audio = np.array(speaker_audio)
            print(f"   Durée totale: {total_speaker_duration:.1f}s ({segment_count} segments)")
            print(f"   Échantillons: {len(speaker_audio):,}")
            
            # Ajouter des statistiques du signal pour ce locuteur
            speaker_rms = np.sqrt(np.mean(speaker_audio**2))
            speaker_max = np.max(np.abs(speaker_audio))
            print(f"   RMS: {speaker_rms:.4f}, Max: {speaker_max:.4f}")
            
            display(Audio(speaker_audio, rate=sr))
        else:
            print(f"   Aucun segment trouvé dans les {max_duration}s")
    
    # 3. Statistiques comparatives
    print(f"\n📊 STATISTIQUES COMPARATIVES SDM:")
    print(f"   Signal complet: {len(audio_array)/sr:.1f}s, {len(audio_array):,} échantillons")
    
    total_segments_duration = sum(end - start for start, end in zip(seg_info['start_times'], seg_info['end_times']))
    coverage = (total_segments_duration / (len(audio_array)/sr)) * 100
    print(f"   Couverture segmentation: {coverage:.1f}%")
    
    signal_rms = np.sqrt(np.mean(audio_array**2))
    print(f"   RMS signal complet: {signal_rms:.4f}")

# Créer les lecteurs audio SDM
create_sdm_audio_players(audio_array, seg_info, audio_analysis['sampling_rate'])

## 6. Statistiques Détaillées

In [ ]:
def detailed_statistics(sample: Dict, audio_analysis: Dict, seg_info: Dict):
    """
    Affiche des statistiques détaillées sur l'échantillon
    """
    print("📊 STATISTIQUES DÉTAILLÉES")
    print("=" * 50)
    
    # Informations générales
    print(f"\n🏷️ MÉTADONNÉES:")
    if 'meeting_id' in sample:
        print(f"   ID de réunion: {sample['meeting_id']}")
    if 'text' in sample:
        text_preview = sample['text'][:200] + "..." if len(sample['text']) > 200 else sample['text']
        print(f"   Aperçu du texte: {text_preview}")
    
    # Statistiques audio
    print(f"\n🎙️ AUDIO:")
    print(f"   Canaux: {audio_analysis['num_channels']}")
    print(f"   Échantillons: {audio_analysis['num_samples']:,}")
    print(f"   Durée: {audio_analysis['duration_seconds']:.2f}s")
    print(f"   Taille: {audio_analysis['num_samples'] * audio_analysis['num_channels'] * 4 / 1024 / 1024:.1f} MB (float32)")
    
    # Statistiques de segmentation
    print(f"\n👥 SEGMENTATION:")
    print(f"   Source: {seg_info['source']}")
    print(f"   Segments totaux: {seg_info['total_segments']}")
    print(f"   Locuteurs uniques: {seg_info['num_speakers']}")
    
    # Durée moyenne par segment
    segment_durations = [end - start for start, end in zip(seg_info['start_times'], seg_info['end_times'])]
    if segment_durations:
        avg_duration = np.mean(segment_durations)
        min_duration = np.min(segment_durations)
        max_duration = np.max(segment_durations)
        print(f"   Durée moy. segment: {avg_duration:.2f}s")
        print(f"   Segment plus court: {min_duration:.2f}s")
        print(f"   Segment plus long: {max_duration:.2f}s")
    
    # Répartition par locuteur
    print(f"\n📈 RÉPARTITION PAR LOCUTEUR:")
    for speaker, count in seg_info['segments_per_speaker'].items():
        # Calculer le temps total pour ce locuteur
        speaker_time = sum(end - start for speaker_seg, start, end in 
                          zip(seg_info['speakers'], seg_info['start_times'], seg_info['end_times'])
                          if speaker_seg == speaker)
        percentage = (speaker_time / seg_info['total_duration']) * 100
        print(f"   {speaker}: {count} segments, {speaker_time:.1f}s ({percentage:.1f}%)")

# Afficher les statistiques
detailed_statistics(sample, audio_analysis, seg_info)

## 7. Test avec d'Autres Échantillons (Optionnel)

In [ ]:
# Charger d'autres échantillons SDM pour comparaison
print("🔄 Test avec d'autres échantillons SDM...")

try:
    # Charger plusieurs échantillons SDM
    dataset_multi = load_ami_sdm(split="train", max_samples=3)
    
    print(f"\n📊 Comparaison de {len(dataset_multi)} échantillons SDM:")
    print("-" * 60)
    
    for i, sample_test in enumerate(dataset_multi):
        audio_test = np.array(sample_test['audio']['array'])
        sr_test = sample_test['audio']['sampling_rate']
        
        # Assurer format mono pour SDM
        if len(audio_test.shape) > 1:
            audio_test = audio_test.flatten()
        
        # Vérifier la segmentation
        has_segments = all(key in sample_test for key in ['segment_start_times', 'segment_end_times', 'segment_speakers'])
        
        print(f"\nÉchantillon SDM {i+1}:")
        print(f"  Audio: {audio_test.shape} @ {sr_test}Hz")
        print(f"  Durée: {len(audio_test) / sr_test:.1f}s")
        print(f"  Configuration: SDM (mono)")
        print(f"  Segmentation: {'✅' if has_segments else '❌'}")
        
        if has_segments:
            num_speakers = len(set(sample_test['segment_speakers']))
            num_segments = len(sample_test['segment_speakers'])
            print(f"  Locuteurs: {num_speakers}, Segments: {num_segments}")
            
            # Statistiques de qualité pour SDM
            audio_rms = np.sqrt(np.mean(audio_test**2))
            audio_snr_estimate = 20 * np.log10(np.std(audio_test) / (np.mean(np.abs(audio_test)) + 1e-10))
            print(f"  Qualité - RMS: {audio_rms:.4f}, SNR estimé: {audio_snr_estimate:.1f}dB")
            
except Exception as e:
    print(f"⚠️  Erreur lors du chargement d'échantillons SDM supplémentaires: {e}")
    print("   Ceci peut être normal si le dataset SDM est limité.")

## ✅ Résumé de la Démonstration SDM

Cette démonstration a montré :

1. **✅ Chargement réussi** du corpus AMI avec configuration SDM (Single Distant Microphone)
2. **✅ Vérification du signal mono** (configuration SDM)
3. **✅ Segmentation des locuteurs** (annotations réelles du corpus AMI)
4. **✅ Visualisation interactive SDM** avec timeline, forme d'onde mono et spectrogramme
5. **✅ Lecture audio** du signal SDM complet et par locuteur
6. **✅ Statistiques détaillées** sur l'échantillon SDM

### 🎯 Points Clés SDM :
- **SDM** = Single Distant Microphone (microphone distant unique)
- Le corpus AMI SDM fournit des données **mono** de haute qualité
- La **segmentation des locuteurs** est disponible dans les annotations
- Configuration **idéale pour tester** les algorithmes de diarisation sur signal mono
- **Signal plus simple** que MDM mais toujours représentatif des conditions réelles

### 📊 Avantages de la configuration SDM :
- **Simplicité** : Un seul canal audio à traiter
- **Efficacité** : Moins de données, traitement plus rapide
- **Réalisme** : Simule des conditions d'enregistrement courantes
- **Compatibilité** : Fonctionne avec tous les algorithmes de diarisation

### 🔗 Ressources Utiles :
- [AMI Corpus Documentation](https://groups.inf.ed.ac.uk/ami/corpus/)
- [Hugging Face Dataset](https://huggingface.co/datasets/edinburghcstr/ami)
- [Configuration SDM Details](https://groups.inf.ed.ac.uk/ami/corpus/setup.shtml)